<a href="https://colab.research.google.com/github/carlosbkm/ADaSci-Agentic-AI-Architect-labs/blob/main/Observing_and_Tracing_Multi_Modal_Multi_Agent_Systems_using_Portkey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library Installation

In [ ]:
!pip install -qU langchain_openai langchain_community tavily-python langchain-groq groq replicate crewai crewai[tools]
!pip install portkey-ai

### API Initialization

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY").strip()
os.environ["REPLICATE_API_TOKEN"] = userdata.get("REPLICATE_API_TOKEN").strip()
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY").strip()
PORTKEY_API_KEY = userdata.get("PORTKEY_API_KEY").strip()

### Web Search Tool Helper Function

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
def web_search_tool(question: str) -> str:
    """This tool is useful when we want web search for current events."""
    websearch = TavilySearchResults()
    response = websearch.invoke({"query":question})
    return response

### Text to Image Creation Helper Function

In [ ]:
import replicate
def text2image(text:str) -> str:
    """This tool is useful when we want to generate images from textual descriptions."""
    output = replicate.run(
    "adirik/flux-cinestill:216a43b9975de9768114644bbf8cd0cba54a923c6d0f65adceaccfc9383a938f",
    input={
        "steps": 28,
        "prompt": text,
        "lora_url": "",
        "control_type": "depth",
        "control_image": "https://replicate.delivery/pbxt/LUSNInCegT0XwStCCJjXOojSBhPjpk2Pzj5VNjksiP9cER8A/ComfyUI_02172_.png",
        "lora_strength": 1,
        "output_format": "webp",
        "guidance_scale": 2.5,
        "output_quality": 100,
        "negative_prompt": "low quality, ugly, distorted, artefacts",
        "control_strength": 0.45,
        "depth_preprocessor": "DepthAnything",
        "soft_edge_preprocessor": "HED",
        "image_to_image_strength": 0,
        "return_preprocessed_image": False
        }
    )
    print(output)
    return output[0]

### Text to Image Processing Helper Function

In [ ]:
def image2text(image_url:str,prompt:str) -> str:
  """This tool is useful when we want to generate textual descriptions from images."""
  # Function
  output = replicate.run(
    "adirik/flux-cinestill:216a43b9975de9768114644bbf8cd0cba54a923c6d0f65adceaccfc9383a938f",
    input={
        "image": image_url,
        "top_p": 1,
        "prompt": prompt,
        "max_tokens": 1024,
        "temperature": 0.2
    }
  )
  return "".join(output)

### Router Tool Setup

In [ ]:
from crewai.tools import tool

@tool("router tool")
def router_tool(question:str) -> str:
  """Router Function"""
  prompt = f"""Based on the Question provide below determine the following:
    1. Is the question directed at generating image ?
    2. Is the question directed at describing the image ?
    3. Is the question a generic one and needs to be answered by searching the web?
    Question: {question}

    RESPONSE INSTRUCTIONS:
    - Answer either 1 or 2 or 3.
    - Answer should strictly be a string.
    - Do not provide any preamble or explanations except for 1 or 2 or 3.

    OUTPUT FORMAT:
    1
    """
  response = llm.invoke(prompt).content.strip()
  if response == "1":
    return 'text2image'
  elif response == "3":
    return 'web_search'
  else:
    return 'image2text'

### Retriever Tool

In [ ]:
@tool("retriver tool")
def retriver_tool(router_response:str,question:str,image_url:str) -> str:
  """Retriver Function"""
  if router_response == 'text2image':
    return text2image(question)
  elif router_response == 'image2text':
    return image2text(image_url,question)
  else:
    return web_search_tool(question)

### Portkey Setup

In [ ]:
from langchain_openai import ChatOpenAI
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL

portkey_headers = createHeaders(
    api_key = PORTKEY_API_KEY,
    # virtual_key = ""
    )

# llm = ChatOpenAI(
#     api_key=PORTKEY_API_KEY,
#     base_url=PORTKEY_GATEWAY_URL,
#     default_headers=portkey_headers
#     )

llm = ChatOpenAI(
    # Set base_url to the Portkey Gateway.
    base_url=PORTKEY_GATEWAY_URL,

    # The ChatOpenAI `api_key` is used to pass the Portkey API Key.
    # Portkey uses this key to authenticate and authorize the request
    # before looking at the model slug for routing/provider key resolution.
    api_key="dummy-key",

    # Pass the Model Catalog slug directly as the model name.
    model="openai/@openai-krol-key/o4-mini-2025-04-16",

    # CRUCIAL: Pass the Portkey-specific headers. This is how Portkey
    # receives your API key and other configurations.
    default_headers=portkey_headers,

    temperature=0.7
)


### Router Agent

In [ ]:
from crewai import Agent
Router_Agent = Agent(
  role = 'Router',
  goal = 'Route user question to a text to image or text to speech or web search',
  backstory = (
    "You are an expert at routing a user question to a text to image or web search."
    "Use the text to image to generate images from textual descriptions."
    "Use the image to text to generate text describing the image based on the textual description."
    "Use the web search to search for current events."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose = True,
  allow_delegation = False,
  llm = llm,
  tools = [router_tool],
)

### Retriever Agent

In [ ]:
Retriever_Agent = Agent(
role = "Retriever",
goal = "Use the information retrieved from the Router to answer the question and image url provided.",
backstory = (
    "You are an assistant for directing tasks to respective agents based on the response from the Router."
    "Use the information from the Router to perform the respective task."
    "Do not provide any other explanation"
),
verbose = True,
allow_delegation = False,
llm = llm,
tools = [retriver_tool],
)

### Router Task

In [ ]:
from crewai import Task
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "If the question {question} instructs to describe a image then use the image url {image_url} to generate a detailed and high quality images covering all the nuances secribed in the textual descriptions provided in the question {question}."
    "Based on the keywords decide whether it is eligible for a text to image or text to speech or web search."
    "Return a single word 'text2image' if it is eligible for generating images from textual description."
    "Return a single word 'image2text' if it is eligible for describing the image based on the question {question} and image url{image_url}."
    "Return a single word 'web_search' if it is eligible for web search."
    "Do not provide any other explaination."
    ),
    expected_output=("Give a choice 'web_search' or 'text2image' or 'image2text' based on the question {question} and image url {image_url}"
    "Do not provide any preamble or explanations except for 'text2image' or 'web_search' or 'image2text'."),
    agent=Router_Agent,
)


### Retriever Task

In [ ]:
retriever_task = Task(
    description=(
        "Use the tool selected by the Router_Agent to either generate an image from text, "
        "describe an image, or search the web. "
        "When the tool generates an image (text2image), you must return ONLY the direct image URL "
        "from the Replicate API response — no markdown, no explanation, no extra text. "
        "If the tool describes an image (image2text), return a short plain text description. "
        "If the tool performs a web search, return a concise textual summary of the results."
    ),
    expected_output=(
        "If the question was to generate an image, return ONLY the direct image URL. "
        "If it was to describe or search, return text only."
    ),
    agent=Router_Agent,
    context=[router_task],
)


### Crew Setup and Initiation

In [ ]:
from crewai import Crew,Process
crew = Crew(
    agents=[Router_Agent,Retriever_Agent],
    tasks=[router_task,retriever_task],
    verbose=True,
)

inputs = {
  "question": "Generate an image based upon this text: a cinematic portrait of a majestic black panther, piercing yellow eyes, moody lighting, soft bokeh, 85mm lens, deep blue jungle background",
  "image_url": " "
}

result = crew.kickoff(inputs=inputs)

### Agent Output -- Image Link

In [ ]:
"""### Agent Output -- Image Link"""

# retriever_task returns ONLY the image URL now
image_url = result.final_output.strip()  # <- use the final output, not .raw
print(image_url)  # optional: quick sanity check

### Printing Image

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

# URL of the image
image_url = result.raw

# Fetch the image
response = requests.get(image_url)

# Check if the request was successful
if response.status_code == 200:
    # Open the image using PIL
    img = Image.open(BytesIO(response.content))

    # Display the image using matplotlib
    plt.imshow(img)
    plt.axis('off')  # Hide the axis
    plt.show()
else:
    print("Failed to retrieve image. Status code:", response.status_code)